In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy.stats.stats import pearsonr   
import matplotlib.pyplot as plt

In [2]:
data = pd.read_excel('full_data_right.xlsx')

In [4]:
import unicodedata

def unicode_to_string(word):
    if pd.isnull(word):
        return 
    else:
        return unicodedata.normalize('NFKD', word).encode('ascii','ignore')
    
def data_to_string(data):
    new_data = data.copy()
    
    col_title_unicode = new_data.columns.values.tolist()
    col_title = map(unicode_to_string,col_title_unicode)
    new_data.columns = col_title
    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == unicode: #check the first not-NaN value
            new_data[col] = map(unicode_to_string,new_data[col])
            
    return new_data

In [5]:
def float_to_int(data):
    new_data = data.copy()    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == np.float64: #check the first not-NaN value
            new_data[col] = new_data[col].round()
    return new_data

In [ ]:
data = data_to_string(data)
data = float_to_int(data)

In [ ]:
data.head()

### We will create a new data frame containing only the variable needed. We drop all the others variables. 

In [31]:
def create_final_data(data):
    new_data = data.copy()
    title = data.columns.tolist()
    dropping_index = []
    keeping_variable = ['Code Insee','% Voix/Ins','% Voix/Exp','C12_POP15P_CS1','C12_POP15P_CS2','C12_POP15P_CS3','C12_POP15P_CS4','C12_POP15P_CS5', 'C12_POP15P_CS6',
                       'C12_POP15P_CS7', 'C12_POP15P_CS8','SNHM12']
    for index, variable in enumerate(title): 
        if variable not in keeping_variable:
            dropping_index.append(index)
    new_data = new_data.drop(new_data.columns[dropping_index], axis = 1)
    return new_data

In [32]:
data2 = create_final_data(data)

In [33]:
data2.head() 

,Code Insee,% Voix/Ins,% Voix/Exp,C12_POP15P_CS1,C12_POP15P_CS2,C12_POP15P_CS3,C12_POP15P_CS4,C12_POP15P_CS5,C12_POP15P_CS6,C12_POP15P_CS7,C12_POP15P_CS8,SNHM12
0,1001,0.00,0.00,15.938462,3.984615,79.692308,87.661538,115.553846,71.723077,167.353846,75.707692,NaN
1,1001,1.20,2.19,15.938462,3.984615,79.692308,87.661538,115.553846,71.723077,167.353846,75.707692,NaN
2,1001,17.12,31.25,15.938462,3.984615,79.692308,87.661538,115.553846,71.723077,167.353846,75.707692,NaN
3,1002,12.05,21.26,0.000000,4.017094,20.085470,52.222222,8.034188,32.136752,64.273504,8.034188,NaN
4,1002,0.00,0.00,0.000000,4.017094,20.085470,52.222222,8.034188,32.136752,64.273504,8.034188,NaN


In [35]:
pearsonr(data2['% Voix/Exp'], data2['C12_POP15P_CS1'])

(nan, 1.0)

In [63]:
data3 = data2.iloc[:5]
data3

,Code Insee,% Voix/Ins,% Voix/Exp,C12_POP15P_CS1,C12_POP15P_CS2,C12_POP15P_CS3,C12_POP15P_CS4,C12_POP15P_CS5,C12_POP15P_CS6,C12_POP15P_CS7,C12_POP15P_CS8,SNHM12
0,1001,0.00,0.00,15.938462,3.984615,79.692308,87.661538,115.553846,71.723077,167.353846,75.707692,NaN
1,1001,1.20,2.19,15.938462,3.984615,79.692308,87.661538,115.553846,71.723077,167.353846,75.707692,NaN
2,1001,17.12,31.25,15.938462,3.984615,79.692308,87.661538,115.553846,71.723077,167.353846,75.707692,NaN
3,1002,12.05,21.26,0.000000,4.017094,20.085470,52.222222,8.034188,32.136752,64.273504,8.034188,NaN
4,1002,0.00,0.00,0.000000,4.017094,20.085470,52.222222,8.034188,32.136752,64.273504,8.034188,NaN


In [93]:
d = data2.copy() 
d = d.dropna()

In [94]:
x = d['% Voix/Exp'].tolist()
y = d['C12_POP15P_CS3'].tolist()

In [95]:
print len(data2), len(d)

88552 12696


In [ ]:
plt.hist(y,bins=np.arange(min(y), max(y) + 10, 10))
plt.show()

In [47]:
x

array([ 15.93846154,  15.93846154,  15.93846154, ...,   0.        ,
         0.        ,   8.2930232 ])

In [45]:
x = scipy.array([-0.65499887,  2.34644428, 3.0])
y = scipy.array([-1.46049758,  3.86537321, 21.0])
pearsonr(x,y)

(0.79617014831975552, 0.41371200873701036)